#  Choropleth Mapping With Folium and PySAL

____
## Author

Serge Rey [sjsrey@gmail.com](sjsrey@gmail)

## Requirements

Since we will be pulling in tiles for basemaps, we need internet connectivity for what follows to work.
____


In [1]:
from IPython.display import HTML
import folium
import pandas as pd
import numpy as np
import pysal as ps
%pylab inline


Populating the interactive namespace from numpy and matplotlib


## Help functions
In order to have folium maps appear in-line we will need two utility functions.


In [2]:
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

The source for these functions is an iPython notebook by [Blake Burkhart](http://nbviewer.ipython.org/gist/bburky/7763555/folium-ipython.ipynb).

## OpenStreet Map Tile with Folium

In [3]:
lat_lon = [33.41715, -111.936517]
map_base = folium.Map(location=lat_lon, zoom_start=11)
map_base.simple_marker(lat_lon, popup='Welcome to PySAL @<b>Coor Hall</b>!')
inline_map(map_base)

## Base layer for south

In [4]:
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=4)
map_osm.geo_json(geo_path = 'data/south.json')
embed_map(map_osm)

### Change the zoom level

In [5]:
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5)
map_osm.geo_json(geo_path = 'data/south.json')
embed_map(map_osm)

## Binding Attribute Data to the Map

In [7]:
import json

f = open(r'data/south.json')
q = json.load(f)
f.close()

In [8]:
type(q)

dict

In [9]:
q.keys()

[u'type', u'features']

In [11]:
q['type']

u'FeatureCollection'

In [12]:
features = q['features']
len(features)

1412

In [13]:
feature_0 = features[0]

In [35]:
feature.keys()

[u'geometry', u'type', u'properties']

In [36]:
feature['geometry'].keys()

[u'type', u'coordinates']

In [14]:
feature_0

{u'geometry': {u'coordinates': [[[-80.6280517578125, 40.39815902709961],
    [-80.60203552246094, 40.480472564697266],
    [-80.62545776367188, 40.504398345947266],
    [-80.6336441040039, 40.53913879394531],
    [-80.6688232421875, 40.568214416503906],
    [-80.66793060302734, 40.58207321166992],
    [-80.63754272460938, 40.61391830444336],
    [-80.61175537109375, 40.619998931884766],
    [-80.57462310791016, 40.615909576416016],
    [-80.52220916748047, 40.63713836669922],
    [-80.52456665039062, 40.47871780395508],
    [-80.52377319335938, 40.4029655456543],
    [-80.6280517578125, 40.39815902709961]]],
  u'type': u'Polygon'},
 u'properties': {u'BLK60': 3.839454752,
  u'BLK70': 3.2554278095,
  u'BLK80': 2.5607402642,
  u'BLK90': 2.5572616581,
  u'CNTY_FIPS': u'029',
  u'COFIPS': 29.0,
  u'DNL60': 6.1681225056,
  u'DNL70': 6.1714993547,
  u'DNL80': 6.1714631077,
  u'DNL90': 6.0508978146,
  u'DV60': 2.2779893943,
  u'DV70': 2.5591397849,
  u'DV80': 5.0619350519,
  u'DV90': 7.2636377

In [37]:
import pandas as pd
indices = []
values = []
coordinates = []
for feature in features:
    indices.append(str(feature['properties']['FIPS']))
    values.append(feature['properties']['HR80'])
    coordinates.append(feature['geometry']['coordinates'])

In [38]:
coordinates

[[[[-80.6280517578125, 40.39815902709961],
   [-80.60203552246094, 40.480472564697266],
   [-80.62545776367188, 40.504398345947266],
   [-80.6336441040039, 40.53913879394531],
   [-80.6688232421875, 40.568214416503906],
   [-80.66793060302734, 40.58207321166992],
   [-80.63754272460938, 40.61391830444336],
   [-80.61175537109375, 40.619998931884766],
   [-80.57462310791016, 40.615909576416016],
   [-80.52220916748047, 40.63713836669922],
   [-80.52456665039062, 40.47871780395508],
   [-80.52377319335938, 40.4029655456543],
   [-80.6280517578125, 40.39815902709961]]],
 [[[-80.52625274658203, 40.16244888305664],
   [-80.58760070800781, 40.17502212524414],
   [-80.65277099609375, 40.178401947021484],
   [-80.67875671386719, 40.1940803527832],
   [-80.65031433105469, 40.245609283447266],
   [-80.6148910522461, 40.276432037353516],
   [-80.60472106933594, 40.306175231933594],
   [-80.60945129394531, 40.373207092285156],
   [-80.62944793701172, 40.38859558105469],
   [-80.6280517578125, 40.3

In [19]:
len(indices)

1412

In [16]:
df = pd.DataFrame({'HR80': values,
                   'FIPS': indices} )

In [21]:
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 legend_name='Homicide Rate HR80')
embed_map(map_osm)

## Using PySAL Map Classification Schemes

In [22]:
y = np.array(df.HR80.tolist())

In [23]:
import pysal as ps

In [24]:
ps.version

'1.11.1'

### Maximum Breaks

In [25]:
max_breaks = ps.Maximum_Breaks(y, 5)

In [26]:
bins = max_breaks.bins.tolist()

In [27]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=bins[:-1],
                 legend_name='Homicide Rate HR80 (Maximum Breaks)')
embed_map(map_osm)

### Quantiles

In [28]:
q5 = ps.Quantiles(y, 5).bins.tolist()

In [29]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=q5[:-1],
                 legend_name='Homicide Rate HR80 (Quintiles)')
embed_map(map_osm)

In [30]:
q6 = ps.Quantiles(y, 6).bins.tolist()

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=q6[:-1],
                 legend_name='Homicide Rate HR80 (Quantiles k=6)')
embed_map(map_osm)

### Fisher-Jenks

In [31]:
fj= ps.Fisher_Jenks(y, 5).bins.tolist()

In [32]:
fjo = ps.Fisher_Jenks(y,5)


In [33]:
fjo.bins
fjo.counts

array([230, 436, 442, 234,  70])

In [34]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=fj[:-1],
                 legend_name='Homicide Rate HR80 (Fisher Jenks)')
embed_map(map_osm)

In [ ]:
ps.esda.mapclassify.Fisher_Jenks_Sampled?